# Assignment 2

Importing Data

In [2]:
import numpy as np
data = np.load('DATA/ct_data.npz')
X_train = data['X_train']; X_val = data['X_val']; X_test = data['X_test']
y_train = data['y_train']; y_val = data['y_val']; y_test = data['y_test']

Question 1a

In [19]:
#defining a function to calculate standard error
def cal_se(data):
    std = np.std(data)
    se = std/ (len(data)**(1/2))
    return se

#verifying the mean on the training set is zero
print(f'The mean on y_train is {round(np.mean(y_train), 8)}.')
print(f'The mean in y_val is {round(np.mean(y_val),8)} +/- {round(cal_se(y_val), 8)}.')

print(f'The mean on the first 5785 entries in y_train is {round(np.mean(y_train[:5785]), 8)} +/- {round(cal_se(y_train[:5785]),8)}.')
#used 8 decimal places as that is the pattern of the data

The mean on y_train is -0.0.
The mean in y_val is -0.21600851 +/- 0.01290338.
The mean on the first 5785 entries in y_train is -0.44247688 +/- 0.01192627.


The standard errors on the mean values of the first 5785 entries of training and the validation dataset suggest that even the edge cases of the mean of the two sets does not meet the overal population mean values. The standard error bars are misleading here because they do not align to what the population mean is??? 

Question 1 b

In [91]:
isConstant = []
isDuplicates = np.array(np.zeros(X_train.shape[1]), dtype='bool') 

#identify all the constant columns
for i in range(X_train.shape[1]) :
    isConstant.append((X_train[:,i] == X_train[0][i]).all())

#identify all the columns which are duplicates to previous columns
for i in range(X_train.shape[1]) :
    for k in range(i+1, X_train.shape[1]) :
        if isDuplicates[k] == False:
            isDuplicates[k] = (X_train[:,i] == X_train[:,k]).all()

#create a list with all the columns to remove
columns_to_remove = np.unique(np.hstack((np.nonzero(isDuplicates)[0], np.nonzero(isConstant)[0])))

#remove these columns and name them modified datasets
X_train_mod = np.delete(X_train, columns_to_remove ,1)
X_val_mod = np.delete(X_val, columns_to_remove ,1)
X_test_mod = np.delete(X_test, columns_to_remove ,1)

#check if the shape still aligns to expectations (number of rows stay the same, number of columns are same for all three sets)
print(X_train_mod.shape, X_val_mod.shape, X_test_mod.shape)

(40754, 373) (5785, 373) (6961, 373)
